In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/bank.csv"
BankData = pd.read_csv(url)
BankData.head(5)

,age,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,y
0,30,married,primary,no,1787,no,no,cellular,79,1,-1,0,unknown,no
1,33,married,secondary,no,4789,yes,yes,cellular,220,1,339,4,failure,no
2,35,single,tertiary,no,1350,yes,no,cellular,185,1,330,1,failure,no
3,30,married,tertiary,no,1476,yes,yes,unknown,199,4,-1,0,unknown,no
4,59,married,secondary,no,0,yes,no,unknown,226,1,-1,0,unknown,no


In [23]:
BankData['y'].unique()  #you can use unique if you would like to find out how many unique attributes each variable have

array(['no', 'yes'], dtype=object)

For dictionary of data please refer to https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

#### Our goal is to define a model best predicts outcome y - success of the marketing campaign. This is the very first a dataset we used to introduce Logistic Regression Models. This time we know few more tricks! In this lecture we are going to use as many techniques as possible on our dataset to make the least possible error! Goal is to use, Logistic Regression, Naive Bayes, Random Forest, KNN, and using VotingClassifier() to combine the results. 

In [24]:
Default_dummy  = pd.get_dummies(BankData['default'], prefix = 'default')
del Default_dummy['default_no']

marital_dummy  = pd.get_dummies(BankData['marital'], prefix = 'marital')
del marital_dummy['marital_married']
del marital_dummy['marital_divorced']

housing_dummy  = pd.get_dummies(BankData['housing'], prefix = 'housing')
del housing_dummy['housing_no']

loan_dummy = pd.get_dummies(BankData['loan'], prefix = 'loan')
del loan_dummy['loan_no']


BankData = pd.concat([BankData,marital_dummy , Default_dummy, housing_dummy, loan_dummy], axis=1)
BankData.head()

,age,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,y,marital_single,default_yes,housing_yes,loan_yes
0,30,married,primary,no,1787,no,no,cellular,79,1,-1,0,unknown,no,0,0,0,0
1,33,married,secondary,no,4789,yes,yes,cellular,220,1,339,4,failure,no,0,0,1,1
2,35,single,tertiary,no,1350,yes,no,cellular,185,1,330,1,failure,no,1,0,1,0
3,30,married,tertiary,no,1476,yes,yes,unknown,199,4,-1,0,unknown,no,0,0,1,1
4,59,married,secondary,no,0,yes,no,unknown,226,1,-1,0,unknown,no,0,0,1,0


In [32]:
X = BankData[['age','balance','duration','campaign','pdays','previous','marital_single','default_yes','housing_yes','loan_yes']]
y = BankData['y']
X.describe()

,age,balance,duration,campaign,pdays,previous,marital_single,default_yes,housing_yes,loan_yes
count,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000
mean,41.562500,1639.671569,272.398284,2.015931,224.865196,3.006127,0.292892,0.006127,0.632353,0.121324
std,11.448836,3006.986113,245.360272,1.581640,117.200417,2.914101,0.455368,0.078086,0.482460,0.326703
min,20.000000,-1400.000000,5.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,33.000000,141.250000,117.750000,1.000000,136.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,39.000000,631.500000,202.000000,1.000000,189.000000,2.000000,0.000000,0.000000,1.000000,0.000000
75%,48.250000,1674.500000,332.000000,2.000000,330.000000,4.000000,1.000000,0.000000,1.000000,0.000000
max,86.000000,26306.000000,1579.000000,11.000000,871.000000,25.000000,1.000000,1.000000,1.000000,1.000000


In [30]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier  #VotingClassifier is part of sklearn.ensemble
from sklearn.neighbors import KNeighborsClassifier

clf1 = LogisticRegression()
clf2 = RandomForestClassifier(max_depth=5, n_estimators = 1000)
clf3 = BernoulliNB()
clf4 = MultinomialNB()
clf5 = GaussianNB()
clf6 = KNeighborsClassifier(n_neighbors=3)

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('bnb', clf3),('gnb',clf5),('knn',clf6)], voting='hard')

In [31]:
for clf in [clf1,clf2,clf3,clf5,clf6,eclf]:
    scores = cross_validation.cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print scores.mean()

0.779489879425
0.778225107745
0.774514107104
0.754911947211
0.747474336025
0.775808805464


In [28]:
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('bnb', clf3),('gnb',clf5)], voting='soft')
for clf in [clf1,clf2,clf3,clf5,eclf]:
    scores = cross_validation.cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print scores.mean()

0.779489879425
0.780671613805
0.774514107104
0.754911947211
0.778278033632
